In [138]:
from pyspark.conf import SparkConf 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
import pyspark
import pyspark.sql.functions as f 
from pymongo import MongoClient

spark = SparkSession.builder.appName("example-pyspark-read-and-write").getOrCreate()

disp = spark.read.parquet("hdfs://localhost:9000/data/coronaAPI")
disp_st = spark.read.parquet("hdfs://localhost:9000/data/coronaStage")

df = disp.select('날짜','area','확진자수')
df = df.withColumn('date',f.to_date(df['날짜']))
df = df.withColumn('date_st',df.date.cast(StringType()))
df = df.drop('날짜','date')

# df.select(df.date.cast(StringType()).alias('ages')).show()
# df = df.date.cast("string")
#df.select()

In [139]:
df1 = disp_st.drop('__index_level_0__')
df = df.join(df1,'area')

In [140]:
df.show()

+----+--------+----------+------+------------------------------+
|area|확진자수|   date_st| Stage|                   Description|
+----+--------+----------+------+------------------------------+
|제주|    2803|2021-09-22|3 단계|   - 제주 전지역 4단계 (21....|
|경남|   10983|2021-09-22|3 단계|   - 경남 전지역 3단계 (21....|
|경북|    7848|2021-09-22|3 단계|  - 경북 일부지역 3단계 (21...|
|전남|    2933|2021-09-22|3 단계|   - 전남 전지역 3단계 (21....|
|전북|    4179|2021-09-22|3 단계|  - 전북 일부지역 3단계 (21...|
|충남|    8627|2021-09-22|3 단계|   - 충남 전지역 3단계 (21....|
|충북|    5906|2021-09-22|3 단계|   - 충북 전지역 3단계 (21....|
|강원|    6172|2021-09-22|3 단계|- 강원 춘천시, 원주시, 동해...|
|경기|   84038|2021-09-22|4 단계|   - 경기 전지역 4단계 (21....|
|세종|    1165|2021-09-22|3 단계|   - 세종 전지역 3단계 (21....|
|울산|    4991|2021-09-22|3 단계|   - 울산 전지역 3단계 (21....|
|대전|    6611|2021-09-22|3 단계|   - 대전 전지역 3단계 (21....|
|광주|    4761|2021-09-22|3 단계|   - 광주 전지역 3단계 (21....|
|인천|   14719|2021-09-22|4 단계|  - 인천 일부지역 4단계 (21...|
|대구|   14687|2021-09-22|3 단계|   - 대구 전지역 3단계 (21....|
|부산| 

In [135]:
for i in df.collect():
   a= i.asDict()
a

{'area': '대구',
 '확진자수': 11830,
 'date_st': '2021-08-03',
 'Stage': '3 단계',
 'Description': '- 대구 전지역 3단계 (21.9.6~10.3.)'}

In [143]:
df.asDict().collect()

AttributeError: 'DataFrame' object has no attribute 'asDict'

In [136]:
## processing_load.py
spark = SparkSession \
    .builder \
    .appName("ojo_test") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017") \
    .config("spark.mongodb.input.database","ojo_db") \
    .config("spark.mongodb.input.collection", "test") \
    .config("packages org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .getOrCreate()
sc =spark.sparkContext

# pymongo connect
client = MongoClient('localhost',27017) # mongodb 27017 port
db = client.ojo_db

for i in df.collect():
    db.test.insert_one(i.asDict())

spark.stop()